# Project By Raunaq Singh Jabbal

Project made considering time constraints, is easy to read and understand, with less LOC's and model has fast inference.

# 0. Imports

In [10]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver         
import time
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import unidecode
from word2number import w2n
import contractions
import regex as re


# 1. Web Scraping and PreProcessing

## A. Ajio Web Scraping

Ajio used as it has an average sized collection (~53K), where I use only women's section, as those apparels have a lot of variety.

Description of most products only has information like dimensions of the product, quantity of the product, etc, hence that information has not been scraped.

Data has been scraped from the below categories:
- Caps & Hats
- Dresses
- Gloves, Scarves & Bandanas
- Jackets & Shrugs
- Jeans & Leggings
- Jumpsuits & Playsuits
- Kurta Suit Sets
- Kurtas & Kurtis
- Leggings
- Lehenga Choli Sets
- Lounge Tops & Sweatshirts
- Maternity Wear 
- Night & Lounge Sets
- Pyjama & Lounge Sets
- Salwars & Churidars
- Sarees
- Shapewear
- Shirts
- Shirts, Tops & Tunics
- Shorts
- Skirts & Ghagras
- Sweaters & SweatShirts
- Trackpants
- trousers and Pants
- Tshirts

Total extracted data: ~6K samples

In [3]:
link = "https://www.ajio.com/search/?query=%3Arelevance%3Agenderfilter%3AWomen%3Al1l3nestedcategory%3AWomen%20-%20Dresses%3Al1l3nestedcategory%3AWomen%20-%20Jackets%20%26%20Shrugs%3Al1l3nestedcategory%3AWomen%20-%20Jeans%20%26%20Jeggings%3Al1l3nestedcategory%3AWomen%20-%20Jumpsuits%20%26%20Playsuits%3Al1l3nestedcategory%3AWomen%20-%20Kurta%20Suit%20Sets%3Al1l3nestedcategory%3AWomen%20-%20Kurtas%20%26%20Kurtis%3Al1l3nestedcategory%3AWomen%20-%20Leggings%3Al1l3nestedcategory%3AWomen%20-%20Lehenga%20Choli%20Sets%3Al1l3nestedcategory%3AWomen%20-%20Lounge%20Tops%20%26%20Sweatshirts%3Al1l3nestedcategory%3AWomen%20-%20Maternity%20Wear%3Al1l3nestedcategory%3AWomen%20-%20Night%20%26%20Lounge%20Sets%3Al1l3nestedcategory%3AWomen%20-%20Pyjamas%20%26%20Lounge%20Shorts%3Al1l3nestedcategory%3AWomen%20-%20Salwars%20%26%20Churidars%3Al1l3nestedcategory%3AWomen%20-%20Sarees%3Al1l3nestedcategory%3AWomen%20-%20Shapewear%3Al1l3nestedcategory%3AWomen%20-%20Shirts%2C%20Tops%20%26%20Tunics%3Al1l3nestedcategory%3AWomen%20-%20Shirts%3Al1l3nestedcategory%3AWomen%20-%20Shorts%3Al1l3nestedcategory%3AWomen%20-%20Skirts%20%26%20Ghagras%3Al1l3nestedcategory%3AWomen%20-%20Sweaters%20%26%20Sweatshirts%3Al1l3nestedcategory%3AWomen%20-%20Track%20Pants%3Al1l3nestedcategory%3AWomen%20-%20Trousers%20%26%20Pants%3Al1l3nestedcategory%3AWomen%20-%20Tshirts%3Al1l3nestedcategory%3AWomen%20-%20Caps%20%26%20Hats%3Al1l3nestedcategory%3AWomen%20-%20Gloves%2C%20Scarves%20%26%20Bandanas&text=clothes&gridColumns=3&segmentIds="
mainlink = "https://www.ajio.com"   # Main link for the website
browser = webdriver.Edge()          # Using Edge browser for Selenium
browser.get(link)                   # Open the link in browser

for i in tqdm(range(0,540000,1000)):                            # Infinite Scroll to get all the products
	browser.execute_script("window.scrollTo(0," + str(i) + ")") # Scroll down the page to get to the bottom
	time.sleep(.3)

100%|██████████| 540/540 [14:45<00:00,  1.64s/it]


In [6]:
for i in tqdm(range(0,540000,1000)):                            # Infinite Scroll to get all the products
	browser.execute_script("window.scrollTo(0,document.body.scrollHeight)") # Scroll down the page to get to the bottom
	time.sleep(.3)

100%|██████████| 540/540 [14:47<00:00,  1.64s/it] 


Title Scraping:

<img src="title1.png" height=400 width=600/>

Link Scraping:

<img src="link1.png" height=400 width=600 />


In [13]:
soup = BeautifulSoup(browser.page_source,'html.parser')             # Get the page source

titles1=[]
links1=[]

for i in soup.find_all('div',class_='nameCls'):                     # Get the titles of the products
    titles1.append(i.text)

for i in soup.find_all('a',class_='rilrtl-products-list__link'):    # Get the links of the products
    links1.append(mainlink+i.attrs['href'])
    
print(len(titles1))
print(len(links1))


titles1 = np.array(titles1).astype("<U241")                                       # Convert to string                    
links1 = np.array(links1).astype("<U241")                                         # Convert to string

6492
6492


In [15]:
browser.close()

## B. ShoppersStop Web Scraping

Good site for extracting apparel information for men (~23K products)

I will extract information about the top 3000 apparel with price range of 1001-2000 from the following categories:

- Ethnic wear
- Jeans
- Shirts
- Shorts
- T-shirts & Polos
- Trousers
- Winter wear

Total extracted data: 3K samples

In [18]:
link2 = "https://www.shoppersstop.com/search/?q=clothes%3Arelevance%3AinStockFlag%3Atrue%3AcategoryName%3AClothing%3Al3category%3AEthnicwear%3Al3category%3AJeans%3Al3category%3AWinterwear%3Aprice%3A1001-2000%3Al3category%3AShirts%3Al3category%3ATrousers%3Al3category%3AShorts%3Al3category%3AT-Shirts%2B%2526%2BPolos&text=clothes&startRange=&endRange=&brandPageId=&linkText=#"
mainlink2 = "https://www.shoppersstop.com"      # Main link for the website
browser2 = webdriver.Edge()                     # Using Edge browser for Selenium
browser2.get(link2)                             # Open the link in browser

In [19]:

for i in tqdm(range(0,15000,1000)):                            					# Infinite Scroll to get all the products
	browser2.execute_script("window.scrollTo(0,document.body.scrollHeight)") 	# Scroll down the page to get to the bottom
	time.sleep(.3)

100%|██████████| 15/15 [00:05<00:00,  2.81it/s]


Title Scraping:

<img src="title2.png" height=400 width=600/>

Link Scraping:

<img src="link2.png" height=400 width=600 />


In [79]:
soup2 = BeautifulSoup(browser2.page_source,'html.parser')               # Get the page source

titles2=[]
links2=[]

for i in soup2.find_all('li',itemtype='http://schema.org/Product'):     # Get the titles of the products
    titles2.append(i.attrs['data-product-name'])

for i in soup2.find_all('div',class_='styleImagesMain'):                # Get the links of the products
    links2.append(mainlink2+i.findChild("a").attrs['href'])

print(len(titles2))
print(len(links2))


titles2 = np.array(titles2).astype("<U241")                             # Convert to have same type as titles1
links2 = np.array(links2).astype("<U241")                               # Converted to have same type as links1

3000
3000


In [93]:
browser2.close()

## C. Raymonds Web Scraping

Another good site for men's fashion, using end of season sale products (~5.5K samples) and selecting the first 3K.

### No href addrss available, yet to find out a way to extract website names, using javascript:void(0), after hovering and clicking, website  for the product opens, can't right click to open in a new tab.

In [94]:
link3= "https://www.myraymond.com/product-list?qid=646712a526fb0f1bbd7ba224&slug=End%20Of%20Reason%20Sale&categoryName=End%20Of%20Reason%20Sale&categoryType=collection"
mainlink3 = "https://www.myraymond.com"      # Main link for the website
browser3 = webdriver.Edge()                     # Using Edge browser for Selenium
browser3.get(link3)                             # Open the link in browser

for i in tqdm(range(0,50000,1000)):                            					# Infinite Scroll to get all the products
	browser3.execute_script("window.scrollTo(0,document.body.scrollHeight)") 	# Scroll down the page to get to the bottom
	time.sleep(.3)

100%|██████████| 50/50 [00:16<00:00,  3.11it/s]


In [109]:
soup3 = BeautifulSoup(browser3.page_source,'html.parser')               # Get the page source

titles3=[]
links3=[]

for i in soup3.find_all('div',class_="content-info-box"):     # Get the titles of the products
    titles3.append(i.children.__next__().text)

for i in soup3.find_all('div',class_='styleImagesMain'):      # Get the links of the products
    links3.append(mainlink3+i.findChild("a").attrs['href'])

titles3 = titles3[::2]                                        # Remove the "ADD TO CART" titles

print(len(titles3))
print(len(links3))

titles3 = np.array(titles3).astype("<U241")                   # Convert to have same type as titles1
links3 = np.array(links3).astype("<U241")                     # Converted to have same type as links1

['Men Slim Fit Light Blue T-Shirt', 'Men Slim Fit Light Blue Shirt', 'Men Slim Fit Light Fawn Shirt', 'Men Slim Fit Light Pink T-Shirt', 'Men Slim Fit Light Blue T-Shirt', 'Men Slim Fit Light Blue T-Shirt', 'Men Slim Fit Blue T-Shirt', 'Men Slim Fit Light Yellow Shirt', 'Men Slim Fit Red T-Shirt', 'Men Slim Fit Green T-Shirt', 'Men Slim Fit Blue T-Shirt', 'Men Slim Fit Pink T-Shirt', 'Men Slim Fit Green T-Shirt', 'Men Slim Fit Pink Shirt', 'Men Slim Fit Grey Shirt', 'Men Slim Fit Green Shirt', 'Men Slim Fit Light Grey T-Shirt', 'Men Slim Fit Black T-Shirt']
18
0


## D. Creating Dataset and Preprocessing

In [92]:
# Preprocessing Function
def preprocess(text):
    text = unidecode.unidecode(text)    # Remove accents
    text = contractions.fix(text)       # Expand contractions
    text = text.lower()                 # Convert to lowercase
    # re.sub(r'\d+', '', text)          # Remove numbers
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    text = text.strip()                 # Remove extra whitespaces
    text = " ".join(text.split())       
    return text

In [ ]:
dataset = pd.DataFrame({'Title':np.hstack((titles1,titles2)),'Link':np.hstack((links1,links2))}) # Create a dataframe
dataset["Title"] = dataset["Title"].apply(preprocess)                                            # Preprocess the titles
dataset.to_csv("dataset.csv",index=False)                                                        # Save the dataset

## 2. Measure Similarity

We need some form of sentence embedding so that we can figure out the most relevant samples from our dataset that match the description given by the user. We also need the most advanced and efficient algorithm to solve this problem.


Talking about BERT, it solves semantic search in a pair wise fashion. The major downside with using BERT here is if we have input given by the user, the titles in our dataset have to be given everytime to the model to compute similarity which is  computationally intensive.

Now, Siamese Networks contain 2 or more identical networks that share the same weight in parallel and are usually used to find out similarity. Siamese networks don't have to train with every possible combination unlike BERT.

Sentence BERT is like the cross-encoder BERT, but it doesn't have the classification/similarity calculation head. It uses a Siamese architecture containing 2 BERT architectures.

We provide 2 sentences, one to each BERT instance, output of which an embedding which is mean pooled. For training, the 2 embeddings are concatenated and passed through a softmax layer for classification. While testing, the cosine similarity of the 2 embeddings is the output.

Sentence Transformers is a Python framework for state of the art sentence, text, and image embedddings based on PyTorch.

- We used a pretained SBERT with the latest "all-mpnet-base-v2" model which outputs a 384 dimensional normalised embedding vector trained on 1 billion training pairs.

- Save the embeddings of the corresponding titles in a csv files so that we don't have to keep recomputing those values.


In [111]:
N=10                                                                            # Number of items to be displayed

model = SentenceTransformer('all-mpnet-base-v2')                                # Loading the Sentence Transformer Model
dataset = pd.read_csv('dataset.csv')                                            # Loading the Datastore from the CSV file
embeddings = model.encode(dataset["Title"], convert_to_tensor=True)             # Getting the embeddings of the Search Query
pd.DataFrame(embeddings.numpy()).to_csv('embeddings.csv', index=False)          # Saving the embeddings in a CSV file

## 3. Function

- Load the dataset including the titles and links and the vector embeddings 
- Preprocess input given by the user
- Use the SBERT model to obtain it's embedding vector
- Calculate cosine similarity of that vector with all the vectors in the embeddings
- Return the N highest links 

In most cases, it takes less than 2 seconds.
We can still improve on this time, by using simpler SBERT pretrained models and using techniquees like PCA as we don't need a 384 dimensional long embedding vector for this problem.

In [122]:
import time
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import unidecode
from word2number import w2n
import contractions
import regex as re
import os 

N=10

def preprocess(text):
    text = unidecode.unidecode(text)    # Remove accents
    text = contractions.fix(text)       # Expand contractions
    text = text.lower()                 # Convert to lowercase
    # re.sub(r'\d+', '', text)          # Remove numbers
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    text = text.strip()                 # Remove extra whitespaces
    text = " ".join(text.split())       
    return text

model = SentenceTransformer('all-mpnet-base-v2')                                # Loading the Sentence Transformer Model

def main(sample):
    t= time.time()
    ajio_embeddings = pd.read_csv('embeddings.csv').to_numpy()                      # Loading the embeddings from the CSV file
    ajio_embeddings = torch.tensor(ajio_embeddings, dtype=torch.float32)            # Converting the embeddings to a tensor
    ajio = pd.read_csv('dataset.csv')                                               # Loading the Datastore from the CSV file

    sample = preprocess(sample)                                                     # Preprocessing the Search Query
    sample_emb = model.encode(sample, convert_to_tensor=True)                       # Getting the embeddings of the Search Query

    scores = np.array(util.cos_sim(sample_emb, ajio_embeddings).tolist()[0])        # Getting the cosine similarity scores of the Search Query with all the items
    indexes = scores.argsort()[::-1][:N]                                            # Getting the indexes of the top N items
    urls= np.array(ajio.iloc[indexes,1])                                            # Getting the links of the top N items
    # print(np.array(list(zip(urls,scores[indexes]))))                                # Printing the links and the scores of the top N items  
    print("Time Taken: ", time.time()-t)
    return urls


In [124]:
main("occasion wear kurta")

[5517 3048 6274 3090 1650 1816 6410 4713 8462 8137]
Time Taken:  1.1728038787841797


array(['https://www.ajio.com/indyes-embroidered-kurti/p/465701505_red',
       'https://www.ajio.com/dream--dzire-embroidered-straight-kurta/p/463720441_white',
       'https://www.ajio.com/agroha-embroidered-straight-kurta/p/461143785_blue',
       'https://www.ajio.com/dream--dzire-embroidered-straight-kurta/p/463723366_green',
       'https://www.ajio.com/fabric-fitoor-embroidered-straight-kurta/p/462283010_red',
       'https://www.ajio.com/cloth-haus-india-embroidered-straight-kurta/p/464897923_blue',
       'https://www.ajio.com/dream--dzire-embroidered-straight-kurta/p/463762953_pink',
       'https://www.ajio.com/true-shape-indian-flared-kurta/p/464192276_yellow',
       'https://www.shoppersstop.com/hangup-jacquard-jacquard-men-s-occasion-wear-kurta-set/p-A21HG87APARROTW/colorChange/A21HG87APARROTW_MULTI?currentPosition=1971&searchQueryUrl=&totalResultVal=6691&searchPageType=category',
       'https://www.shoppersstop.com/hangup-jacquard-jacquard-mens-occasion-wear-kurta-set/p